In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import text
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
import dask.dataframe as dd
from datetime import datetime
from csv import DictReader
from math import exp, log, sqrt
from random import random,shuffle
import pickle
import sys
from ngram import getUnigram
import string
import nltk
from nltk.util import ngrams # function for making ngrams
import re


ModuleNotFoundError: No module named 'ngram'

In [ ]:
def prepare_bigram(path,out):
    print(path)
    c = 0
    start = datetime.now()
    with open(out, 'w') as outfile:
        outfile.write('name_bigram,item_desc_bigram\n')
        for t, row in enumerate(DictReader(open(path), delimiter=',')): 
            if c%100000==0:
                print('finished',c)
            
            a1 = remove_punctuation(str(row['n_p']).lower()).split(' ')
            a2 = remove_punctuation(str(row['ide_p']).lower()).lower().split(' ')
            
            a1_bigram = ngrams(a1, 2)
            a2_bigram = ngrams(a2, 2)
            if c==0:
                print('----')
                print(row['n_p'])
                print(row['ide_p'])
                print(list(a1_bigram))
                print('----')
            a1_bigram = ' '.join(list(a1_bigram))
            a2_bigram = ' '.join(list(a2_bigram))
            if c==0:
                print(list(a1_bigram))
                print('-----------------')
            outfile.write('%s,%s\n' % (a1_bigram, a2_bigram))
            c+=1
        end = datetime.now()



In [24]:
import numpy as np
import math
 
def signature_bit(data, planes):
    """
    LSH signature generation using random projection
    Returns the signature bits for two data points.
    The signature bits of the two points are different
    only for the plane that divides the two points.
    """
    sig = 0
    for p in planes:
        sig <<=  1
        if np.dot(data, p) >= 0:
            sig |= 1
    return sig
 
def bitcount(n):
    """
    gets the number of bits set to 1
    """
    count = 0
    while n:
        count += 1
        n = n & (n-1)
    return count

def length(v):
    """returns the length of a vector"""
    return math.sqrt(np.dot(v, v))
 
if __name__ == '__main__':
    dim = 200000       # dimension of data points (# of features)
    bits = 1024    # number of bits (planes) per signature
    run = 10       # number of runs
    avg = 0

In [26]:
%%time
for r in range(run):
    # Generate two data points p1, p2
    pt1 = np.random.randn(dim)
    pt2 = np.random.randn(dim)

    # reference planes as many as bits (= signature bits)
    ref_planes = np.random.randn(bits, dim)

    # signature bits for two data points
    sig1 = signature_bit(pt1, ref_planes)
    sig2 = signature_bit(pt2, ref_planes)

    # Calculates exact angle difference
#     cosine = np.dot(pt1,pt2)/length(pt1)/length(pt2)
#     exact = 1 - math.acos(cosine)/math.pi

    # Calculates angle difference using LSH based on cosine distance
    # It's using signature bits' count
    cosine_hash = 1 - bitcount(sig1^sig2)/bits
    print(cosine_hash)
    # Difference between exact and LSH
#     diff = abs(cosine_hash-exact)/exact
#     avg += diff
#     print(exact, cosine_hash, diff)
#     print('exact %.3f, hash %.3f, diff %.3f') %(exact, cosine_hash, diff)

# print('avg diff = {}'.format(avg/run))

0.49609375
0.4833984375
0.462890625
0.50390625
0.4736328125
0.478515625
0.501953125
0.494140625
0.5
0.509765625
CPU times: user 1min 37s, sys: 2.02 s, total: 1min 39s
Wall time: 1min 19s


In [2]:
import pyximport
pyximport.install()

(None, <pyximport.pyximport.PyxImporter at 0x7f74e2df2828>)

In [12]:
from my_minhash import *

In [4]:
import pandas as pd
data = '../data/'
cache = '../cache/'

train = pd.read_csv(data+"train.tsv", sep='\t')
test = pd.read_csv(data+"test.tsv", sep='\t')

In [5]:
test.head()

,test_id,name,item_condition_id,category_name,brand_name,shipping,item_description
0,0,"Breast cancer ""I fight like a girl"" ring",1,Women/Jewelry/Rings,NaN,1,Size 7
1,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers",1,Other/Office supplies/Shipping Supplies,NaN,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers Lined..."
2,2,Coach bag,1,Vintage & Collectibles/Bags and Purses/Handbag,Coach,1,Brand new coach bag. Bought for [rm] at a Coac...
3,3,Floral Kimono,2,Women/Sweaters/Cardigan,NaN,0,-floral kimono -never worn -lightweight and pe...
4,4,Life after Death,3,Other/Books/Religion & Spirituality,NaN,1,Rediscovering life after the loss of a loved o...


In [6]:
test_id = test.test_id.values

In [7]:
price = train['price'].values

In [8]:
del train['price']

In [9]:
df_all = pd.concat([train, test], axis=0)

In [55]:
df_all['n_id'] = df_all['name'].astype(str) + '+++++_____+++++' + df_all['item_description'].astype('str')

In [43]:
df_all.head()

,brand_name,category_name,item_condition_id,item_description,name,shipping,test_id,train_id,n_id
0,NaN,Men/Tops/T-shirts,3,No description yet,MLB Cincinnati Reds T Shirt Size XL,1,NaN,0.0,MLB Cincinnati Reds T Shirt Size XL__No descri...
1,Razer,Electronics/Computers & Tablets/Components & P...,3,This keyboard is in great condition and works ...,Razer BlackWidow Chroma Keyboard,0,NaN,1.0,Razer BlackWidow Chroma Keyboard__This keyboar...
2,Target,Women/Tops & Blouses/Blouse,1,Adorable top with a hint of lace and a key hol...,AVA-VIV Blouse,1,NaN,2.0,AVA-VIV Blouse__Adorable top with a hint of la...
3,NaN,Home/Home Décor/Home Décor Accents,1,New with tags. Leather horses. Retail for [rm]...,Leather Horse Statues,1,NaN,3.0,Leather Horse Statues__New with tags. Leather ...
4,NaN,Women/Jewelry/Necklaces,1,Complete with certificate of authenticity,24K GOLD plated rose,0,NaN,4.0,24K GOLD plated rose__Complete with certificat...


In [14]:
%%time
data1 = ['minhash', 'is', 'a', 'probabilistic', 'data', 'structure', 'for',
        'estimating', 'the', 'similarity', 'between', 'datasets']
data2 = ['minhash', 'is', 'a', 'probability', 'data', 'structure', 'for',
        'estimating', 'the', 'similarity', 'between', 'documents']

m1, m2 = MinHash(), MinHash()
for d in data1:
    m1.update(d.encode('utf8'))
for d in data2:
    m2.update(d.encode('utf8'))
print("Estimated Jaccard for data1 and data2 is", m1.jaccard(m2))


Estimated Jaccard for data1 and data2 is 0.7109375
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 4.27 ms


In [31]:
%load_ext cython

In [32]:
%%cython
def cyjaccard1(seq1, seq2):
    cdef set set1 = set(seq1)
    cdef set set2 = set()

    cdef Py_ssize_t length_intersect = 0

    for char in seq2:
        if char not in set2:
            if char in set1:
                length_intersect += 1
            set2.add(char)

    return 1 - (length_intersect / float(len(set1) + len(set2) - length_intersect))

In [36]:
import pyximport
pyximport.install()
import my_cyjaccard

In [27]:
# from cyjaccard import *

In [35]:
%%time
data1 = ['minhash', 'is', 'a', 'probabilistic', 'data', 'structure', 'for',
        'estimating', 'the', 'similarity', 'between', 'datasets']
data2 = ['minhash', 'is', 'a', 'probability', 'data', 'structure', 'for',
        'estimating', 'the', 'similarity', 'between', 'documents']

print("Estimated Jaccard for data1 and data2 is", 1-cyjaccard1(data1, data2))

Estimated Jaccard for data1 and data2 is 0.7142857142857143
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 475 µs


In [38]:
%%time
data1 = ['minhash', 'is', 'a', 'probabilistic', 'data', 'structure', 'for',
        'estimating', 'the', 'similarity', 'between', 'datasets']
data2 = ['minhash', 'is', 'a', 'probability', 'data', 'structure', 'for',
        'estimating', 'the', 'similarity', 'between', 'documents']

print("Estimated Jaccard for data1 and data2 is", 1-my_cyjaccard.cyjaccard2(data1, data2))

Estimated Jaccard for data1 and data2 is 0.7142857142857143
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 421 µs


In [56]:
def jaccard_c(x):
#     print('---------------------------------')
#     print(x)
    x1, x2 = x.split('+++++_____+++++')
    x1_tokens = x1.split(' ')
    x2_tokens = x2.split(' ')
    j = 1-my_cyjaccard.cyjaccard2(x1_tokens, x2_tokens)
    return j

In [57]:
%%time
df_all['j_n_id'] = df_all['n_id'].map(lambda x: jaccard_c(str(x)))

CPU times: user 12.2 s, sys: 32 ms, total: 12.2 s
Wall time: 12.2 s


In [58]:
df_all.head()

,brand_name,category_name,item_condition_id,item_description,name,shipping,test_id,train_id,n_id,j_n_id
0,NaN,Men/Tops/T-shirts,3,No description yet,MLB Cincinnati Reds T Shirt Size XL,1,NaN,0.0,MLB Cincinnati Reds T Shirt Size XL+++++_____+...,0.000000
1,Razer,Electronics/Computers & Tablets/Components & P...,3,This keyboard is in great condition and works ...,Razer BlackWidow Chroma Keyboard,0,NaN,1.0,Razer BlackWidow Chroma Keyboard+++++_____++++...,0.029412
2,Target,Women/Tops & Blouses/Blouse,1,Adorable top with a hint of lace and a key hol...,AVA-VIV Blouse,1,NaN,2.0,AVA-VIV Blouse+++++_____+++++Adorable top with...,0.000000
3,NaN,Home/Home Décor/Home Décor Accents,1,New with tags. Leather horses. Retail for [rm]...,Leather Horse Statues,1,NaN,3.0,Leather Horse Statues+++++_____+++++New with t...,0.030303
4,NaN,Women/Jewelry/Necklaces,1,Complete with certificate of authenticity,24K GOLD plated rose,0,NaN,4.0,24K GOLD plated rose+++++_____+++++Complete wi...,0.000000
